In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConstantInput(nn.Module):
    def __init__(self, file, size, fixed):                
        super().__init__()
        t = torch.load(file)
        t = F.interpolate(t, size=size, mode='bilinear', align_corners=True)        
        self.fixed = fixed
        if fixed:
            self.register_buffer('input', t)
        else:
            self.input = nn.Parameter(t)

    def forward(self, t):
        if self.fixed:
            return self.input.expand(t.shape[0], -1, -1, -1)
        return self.input.repeat(t.shape[0], 1, 1, 1)
    
file = './data/mean_face_256.pth'
size = 8
fixed = True
ci = ConstantInput(file, size, fixed)
ci(torch.rand(5, 3, size, size)).shape

torch.Size([5, 3, 8, 8])

In [2]:
slices = torch.tensor([
    [2, 4],
    [3, 3],
    [1, 2],
])
slices, slices.dtype

(tensor([[2, 4],
         [3, 3],
         [1, 2]]),
 torch.int64)

In [3]:
sz = 4
res = torch.zeros(slices.size(0), 3, sz, sz)
res.shape

torch.Size([3, 3, 4, 4])

In [12]:
initial = ci.input[0]
for i, (r, c) in enumerate(slices):
    print(r, c)Michael Zarren
    print(initial[:, r:r+sz, c:c+sz].shape)
    res[i] = initial[:, r:r+sz, c:c+sz]
res.shape    

tensor(2) tensor(4)
torch.Size([3, 4, 4])
tensor(3) tensor(3)
torch.Size([3, 4, 4])
tensor(1) tensor(2)
torch.Size([3, 4, 4])


torch.Size([3, 3, 4, 4])

In [6]:
initial.shape

torch.Size([3, 8, 8])

In [10]:
initial[:, r:r+sz, c:c+sz].shape

torch.Size([3, 4, 4])

In [15]:
class Slices2D(nn.Module):
    def __init__(self, file, size):                
        super().__init__()
        t = torch.load(file)
        t = F.interpolate(t, size=size, mode='bilinear', align_corners=True)[0]
        self.register_buffer('t', t)

    def forward(self, slices, size):
        res = torch.empty(slices.size(0), 3, size, size, device=slices.device)
        for i, (r, c) in enumerate(slices):
            res[i] = initial[:, r:r+size, c:c+size]
        return res

s2d = Slices2D(file, size)    
s2d(slices, 4).shape

torch.Size([3, 3, 4, 4])